In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

## Part 1

# Forward Simulation

In [56]:
def sim(n=1000, T=1000, alpha = 0.9, beta = 0.2, gamma = 0.1, lamb0=1, lamb1=5):
    #Defining the three possible values of C
    ck = np.arange(3)   
    
    #Gamma matrix for assigning P(C_t | C_{t-1})
    Gamma = np.array([[1-gamma, 0, gamma], 
                      [0, 1-gamma, gamma], 
                      [beta/2, beta/2, 1-beta]])
    
    #Creating random variables with probabilities based of the gamma matrix
    C_transition = [
        stats.rv_discrete(values=(ck,Gamma[0,])),#P(C_t |C_{t-1} = 0)
        stats.rv_discrete(values=(ck,Gamma[1,])),#P(C_t |C_{t-1} = 1)
        stats.rv_discrete(values=(ck,Gamma[2,])),#P(C_t |C_{t-1} = 2)
    ]

    #Creating output vector of C's
    C = np.zeros(T, np.int64)
    #Initializing the C vector
    C[0] = 2

    #Filling up the C-vector with values
    for i in range(T-1):
        C[i+1] = C_transition[C[i]].rvs()
    
    #CPT of Z
    Z_given_C = np.array([1-alpha, alpha, 0.5])  #P(Z = 1| C =c)
    
    #Initializing Z. size=[n,T] specifies we need to create n copies of a series of T simulations
    Z = stats.bernoulli(Z_given_C[C]).rvs(size=[n,T])
    #input:
    '''
    [C_1, C_2, ... C_T]
    '''
    #output: 
    '''[   
        [Z_11, Z_21, Z_31, ... , Z_T1]
        [Z_12, Z_22, Z_32, ... , Z_T2]
        ...
        [Z_1n, Z_2n, Z_3n, ... , Z_Tn]
    ]'''

    #Initialize X   
    X = stats.poisson(np.where(Z, lamb1, lamb0)).rvs()
    #input
    '''[   
        [f(Z_11), f(Z_21), f(Z_31), ... , f(Z_T1)]
        [f(Z_12), f(Z_22), f(Z_32), ... , f(Z_T2)]
        ...
        [f(Z_1n), f(Z_2n), f(Z_3n), ... , f(Z_Tn)]
        ]    
        where
        f(z) = lamb0+(lamb1-lamb0)*z
    '''

    #output
    '''[   
        [X_11, X_21, X_31, ... , X_T1]
        [X_12, X_22, X_32, ... , X_T2]
        ...
        [X_1n, X_2n, X_3n, ... , X_Tn]
    ]'''

    return C,Z,X


C, Z, X = sim(n = 2,T=10)

# Visualizing